In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, average_precision_score, accuracy_score, confusion_matrix

import xgboost as xgb
import csv
import pandas as pd
from random import seed
import matplotlib.pyplot as plt 
import numpy as np
import pickle
SEED = 12345
seed(SEED)

In [6]:
df_planes = pd.read_csv('training_data.csv')
df_planes['squawk'] = df_planes['squawk']
features = list(set(df_planes.columns) - set(["icao_hex", "nnum", "png_filename", "start_time", "end_time", "is_hover", "record_cnt"]))

XGBoost gets like 82.5% AUPR compared to 71% for Random Forests so that's cool

In [46]:
auprs = []
aupr_bls = []
clf = None

param = {'max_depth': 3, 'eta': 0.1, 'verbosity': 0}
# {'max_depth': 3, 'eta': 0.1} gets 82.5
# param['nthread'] = 4
param['eval_metric'] = 'aucpr'
num_round = 200
for i in range(5): 
    train_df, test_df = train_test_split(df_planes, 
                                         test_size=0.2, 
                                         shuffle=True,
                                         random_state=SEED+i
                                        )

    dtrain = xgb.DMatrix( train_df[features], label=train_df[['is_hover']])
    dtest = xgb.DMatrix( test_df[features], label=test_df[['is_hover']])
    
    evallist = [(dtrain, 'train')]
    bst = xgb.train(param, dtrain, num_round, [])
    
    ypred = bst.predict(dtest)
    aupr = average_precision_score(test_df[['is_hover']], ypred)
    auprs.append(aupr)
    aupr_baseline = float(len([a for a in test_df["is_hover"] if a]))/len(test_df["is_hover"])
    aupr_bls.append(aupr_baseline)


print("AUPR: {} | baseline: {}".format(sum(auprs) / len(auprs), sum(aupr_bls) / len(aupr_bls) ))
print()

AUPR: 0.8245340924417659 | baseline: 0.2130718954248366



In [ ]:
from IPython.display import Image, display

for row, pred in zip(test_df.values, predicted_labels):
    if pred != row[5]:
        print(row[2])
        print("actual: {}; predicted: {}".format(row[5], pred))
        display(Image(row[2], width=400))
